# Importing all necessary libraries

In [2]:
import os
from pathlib import Path
import cv2
import geojson
import rasterio
import rasterio.mask
from rasterio.enums import Resampling
from rasterio.features import rasterize
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import unary_union
import numpy as np
import matplotlib.pyplot as plt

# Converting Tif files to PNG and saving them 

In [ ]:
path = r'' #Folder where Tif files are located
dest = r'' #Folder where to save PNG files

for files in os.listdir(path):
    name = Path(files).stem
    name = name + '.png'
    with rasterio.open(os.path.join(path, files)) as src:
        data = src.read(
            out_shape=(src.count, int(src.height), int(src.width)),
            resampling = Resampling.bilinear
        )
        transform = src.transform

    with rasterio.open(os.path.join(dest,name),'w',driver='PNG', height=data.shape[1], width=data.shape[2],
                       count=src.count, dtype = data.dtype) as dst:
        dst.write(data)

# Adding Tif (raster) and Geojson (shape) file to the 2 different lists

In [ ]:
rasters_path = r'' #Folder where Tif files are located
shapes_path = r'' # Folder where Geojson files are located


rasters_list = []
shapes_list = []

for raster in os.listdir(rasters_path):
    raster_list_path = os.path.join(rasters_path, raster)
    rasters_list.append(raster_list_path)

for shape in os.listdir(shapes_path):
    shape_list_path = os.path.join(shapes_path, shape)
    shapes_list.append(shape_list_path)

# Using Tif and Geojson to create binary mask and then convert it to Yolov8 format (txt file)

In [ ]:
n = len(shapes_list)

for i in range(n):
    raster_path = rasters_list[i]
    shape_path = shapes_list[i]
    with open(shape_path, 'r') as file:
        gjs_dt = geojson.load(file)

    #Checking whether there is object(s) (in our case house(s)) in Geojson file:

    if len(gjs_dt['features']) == 0: #if not creating empyt txt file
        save_path = r'' #Where to save all txt files
        name = Path(raster_path).stem
        with open('{}.txt'.format(os.path.join(save_path, name)), 'w') as f:
            pass
        
        
    else: #if yes first deleting zeros in third position in geojson files and saving
        for feat in gjs_dt['features']:
            if (feat['geometry']['type'] == 'MultiPolygon'):
                    for a in feat['geometry']['coordinates']:
                        for b in a:
                            for c in b:
                                del c[2]
            else:
                for j in feat['geometry']['coordinates']:
                    for q in j:
                        del q[2]

        with open(shape_path, 'w') as file:
            geojson.dump(gjs_dt, file)
        with rasterio.open(raster_path, "r") as src:    
            raster_img = src.read()
            raster_meta = src.meta

        train_df = gpd.read_file(shape_path)

        #Making binary mask by using Tif and Geojson files
        def poly_from_utm(polygon, transform):
            poly_pts = []
    
            poly = unary_union(polygon)
            for x in np.array(poly.exterior.coords):
        
                # Convert polygons to the image CRS
                poly_pts.append(~transform * tuple(x))
        
            # Generate a polygon object
            new_poly = Polygon(poly_pts)
            return new_poly

        # Generate Binary maks

        poly_shp = []
        im_size = (src.meta['height'], src.meta['width'])
        for num, row in train_df.iterrows():
            if row['geometry'].geom_type == 'Polygon':
                poly = poly_from_utm(row['geometry'], src.meta['transform'])
                poly_shp.append(poly)
            else:
                for p in row['geometry'].geoms:
                    poly = poly_from_utm(p, src.meta['transform'])
                    poly_shp.append(poly)

        mask = rasterize(shapes=poly_shp, 
                        out_shape=im_size)

        mask = mask.astype("uint16")
        rel = r'' #Where to save all binary masks (in Tif format)
        sving = Path(raster_path).stem
        sving_tif = sving + '.tif'
        save_path = os.path.join(rel,sving_tif)
        bin_mask_meta = src.meta.copy()
        bin_mask_meta.update({'count': 1})
        with rasterio.open(save_path, 'w', **bin_mask_meta) as dst:
            dst.write(mask * 255, 1)

        with rasterio.open(save_path) as src:
            data = src.read(
                out_shape=(src.count, int(src.height), int(src.width)),
                resampling = Resampling.bilinear
            )
            transform = src.transform
        rel2 = r'' #Where to save all binary masks (in PNG format)
        sving_png = sving + '.png'
        with rasterio.open(os.path.join(rel2, sving_png),'w',driver='PNG', height=data.shape[1], width=data.shape[2], count=src.count, dtype = data.dtype) as dst:
            dst.write(data)

# Converting binary masks (PNG) to Yolov8 format (txt files)

In [ ]:
input_dir = r'' #Path where you saved all binary masks (PNG)
output_dir = r'' #Path where you saved txt files (labels)

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 0:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()